### Looking at lexically-redacted examples

I'm interested in finding some **fully lexically-redacted examples** from the test set.

In [1]:
import sys
sys.path.append('/home/jxm3/research/deidentification/unsupervised-deidentification')

In [2]:
from dataloader import WikipediaDataModule
import os

num_cpus = len(os.sched_getaffinity(0))

dm = WikipediaDataModule(
    document_model_name_or_path='roberta-base',
    profile_model_name_or_path='google/tapas-base',
    dataset_name='wiki_bio',
    dataset_train_split='train[:256]',
    dataset_val_split='val[:256]',
    dataset_test_split='test[:100%]',
    dataset_version='1.2.0',
    num_workers=num_cpus,
    train_batch_size=256,
    eval_batch_size=256,
    max_seq_length=128,
    sample_spans=False,
)
dm.setup("fit")

Initializing WikipediaDataModule with num_workers = 8 and mask token `<mask>`
loading wiki_bio[1.2.0] split train[:256]
loading wiki_bio[1.2.0] split val[:256]
loading wiki_bio[1.2.0] split test[:100%]
                        

In [11]:
dm.test_dataset['profile'][33].split()

['fullname',
 '||',
 'nikolai',
 'andreyevich',
 'chizhevsky',
 'name',
 '||',
 'nikolai',
 'chizhevsky',
 'caps',
 '||',
 '0',
 '21',
 'position',
 '||',
 'defender',
 'pcupdate',
 '||',
 '18:47',
 ',',
 '30',
 'january',
 '2014',
 '-lrb-',
 'utc',
 '-rrb-',
 'years',
 '||',
 '2011-2012',
 '2013',
 'clubs',
 '||',
 'fc',
 'shinnik',
 'yaroslavl',
 'fc',
 'znamya',
 'truda',
 'orekhovo-zuyevo',
 'birth_date',
 '||',
 '15',
 'march',
 '1993',
 'article_title',
 '||',
 'nikolai',
 'chizhevsky',
 'goals',
 '||',
 '0',
 '1',
 'height',
 '||',
 '1.74']

In [9]:
dm.test_dataset['document'][33].split()

['he',
 'made',
 'his',
 'debut',
 'in',
 'the',
 'russian',
 'second',
 'division',
 'for',
 'fc',
 'znamya',
 'truda',
 'orekhovo-zuyevo',
 'on',
 'july',
 '18',
 ',',
 '2013',
 'in',
 'a',
 'game',
 'against',
 'fc',
 'kolomna',
 '.']

In [12]:
dm.test_dataset['document'][34].split()

['darcy',
 'james',
 'argue',
 'is',
 'a',
 'vancouver-born',
 ',',
 'brooklyn-based',
 'jazz',
 'composer',
 'and',
 'bandleader',
 '.',
 'he',
 'is',
 'best',
 'known',
 'for',
 'his',
 'work',
 'with',
 'his',
 '18-piece',
 'ensemble',
 ',',
 'secret',
 'society',
 ',',
 'with',
 'which',
 'he',
 'has',
 'released',
 'two',
 'grammy-nominated',
 'albums',
 '.']

In [14]:
for i, doc in enumerate(dm.test_dataset['document_redact_lexical'][:150]):
    print(i, doc)
    print()

0 leonard shenoff <mask> ( born <mask> <mask> , <mask> ) is a former major league baseball player .
he was the first-round pick of the <mask> <mask> <mask> the secondary phase of the <mask> 1970 major league baseball draft , tenth overall .


1 <mask> <mask> ( born <mask> <mask> <mask> in rhèges ) is a member <mask> the <mask> <mask> <mask> .
he was first elected in <mask> , and represents the <mask> department .
a <mask> by profession , he serves as an independent , and also serves as the head <mask> the <mask> <mask> <mask> <mask> , to which he was elected to represent the <mask> <mask> <mask> in 1980 .
in 1998 and 2008 , he was re-elected to the <mask> in the first round , avoiding the need for a run-off vote .
having contributed to the creation <mask> the university <mask> technology <mask> troyes , in 1998 he was made the first vice <mask> <mask> the university board , <mask> which he is currently the <mask> .
he is a member <mask> the <mask> 's committee on the laws relating to t

In [175]:
from IPython.display import HTML, display
import html

wrap_th = lambda s: f'<th>{s}</th>'
wrap_td = lambda s: f'<td>{s}</td>'

def table_from_table_rows(rows_str: str) -> List[Tuple[str, str]]:
    return [[el.strip() for el in r.split('||')] for r in rows_str.split('\n')]

def html_format_doc(doc: str) -> str:
    # return html.escape(doc)
    return doc.replace('<mask>', '<span style="color: darkgray; background-color: darkgray; line-height: 16px; font-size: 14px; font-weight: 100"> &lt;mask&gt;</span>')

def make_prof_html(profile: str) -> str:
    table = table_from_table_rows(profile)
    s = '<table style="border: 1px solid black"><tbody>'
    # print('table:', table)
    for rkey, rval in table:
        s += '<tr>'
        s += f'<th><b>{rkey}</b></th>'
        s += f'<td>{rval}</td>'
        s += '</tr>'
    s += '</tbody></table>'
    return s

table = '<table>'
table += f'<tr>{wrap_th("ID")}{wrap_th("Name")}{wrap_th("Redacted document")}{wrap_th("Profile")}</th>'
for i in range(100):
    table += '<tr>'
    ex = correct_lex_redacted_docs[i]
    table += wrap_td(ex['text_key_id'])
    table += wrap_td(ex['name'])
    table += wrap_td(html_format_doc(ex['document_redact_lexical']))
    table += wrap_td(make_prof_html(ex['profile']))
    table += '</tr>'
table += '</table>'
    
display(HTML(table))

In [33]:
[doc for doc in dm.val_dataset['document'] if 'table tennis' in doc]

['hui jun is a male former table tennis player from china .\n',
 'cornelia molnar ( born 26 november 1983 ) is a croatian table tennis player .\nshe competed for croatia at the 2004 summer olympics and 2012 summer olympics .\n',
 'bora vang ( 王博 , born april 9 , 1987 ) is a chinese-born turkish national table tennis player .\nthe tall athlete at competes for adana table tennis club , where he is coached by sabahattin sabrioğlu .\nhe won the gold medal in the mized doubles event at the 2010 european mixed double championships held in subotica , serbia together with his turkish teammate Şirin he .\nin 2012 , he and his turkish teammate melek hu won the silver medal at the european mixed double championships held in buzau , romania .\nvang qualified for the 2012 summer olympics after the world qualification championship held in doha , qatar .\nat the 2013 mediterranean games held in mersin , turkey , he won the silver medal in the singles event .\n',
 "mikael appelgren ( born october 15 ,

In [59]:
len([doc for doc in dm.val_dataset['document'] if 'brazil' in doc])

207

In [61]:
len([doc for doc in dm.val_dataset['document'] if (('wales' in doc) or ('welsh' in doc)) and ('actor' in doc)])

10

In [72]:
def get_num_words_until_paren(s: str) -> int:
    try:
        return s.split(' ').index('(')
    except ValueError:
        return -1

    
four_word_named_people = [(i, doc) for i,doc in enumerate(dm.val_dataset['document']) if (get_num_words_until_paren(doc) == 4) and lex_pred_correct[i]]
len(four_word_named_people)

579

In [75]:
brazilian_four_word_named_people = [(i, doc) for i, doc in four_word_named_people if (('brazil' in doc) or ('brasil' in doc))]
len(brazilian_four_word_named_people)

26

In [76]:
brazilian_four_word_named_people[0]

(82,
 'thiago dos santos costa ( born february 28 , 1983 ) is a brazilian footballer who plays for são luiz .\n')

In [77]:
brazilian_four_word_named_people[1]

(460,
 'luis ricardo silva umbelino ( born 21 january 1984 ) is a brazilian footballer who plays for botafogo , on loan from são paulo fc .\nmainly a right back , he can perform equally as a right midfielder .\n')

In [78]:
brazilian_four_word_named_people[2]

(493,
 'fabián raphael estay silva ( born october 5 , 1968 ) is a chilean football midfielder .\nhe was capped 69 times and scored five goals for the chile national team between 1990 and 2001 , including four games at the 1998 fifa world cup .\nestay made his debut for the senior national squad on october 17 , 1990 in a friendly against brazil .\nfabián played for club sides in chile , switzerland , greece , mexico and colombia .\n')

In [79]:
brazilian_four_word_named_soccer_players = [(i, doc) for i,doc in brazilian_four_word_named_people if 'play' in doc]
len(brazilian_four_word_named_soccer_players)

19

In [83]:
len([doc for doc in dm.val_dataset['document'] if (('iran' in doc) and ('volleyball' in doc))])

1

In [93]:
import glob
import pandas as pd

adv_csv_path = glob.glob('../adv_csvs/model_8_1day/*')[0]

df = pd.read_csv(adv_csv_path)
df['perturbed_text'] = df['perturbed_text'].map(lambda s: s.replace('<SPLIT>', '\n'))

In [102]:
def make_html_table(rows: List[List[str]]) -> str:
    assert len(rows) > 0
    table = '<table style="font-size: 18px">'
    table += '<tr>'
    table += ''.join([wrap_th(el) for el in rows[0]])
    table += '</tr>'
    for row in rows[1:]:
        table += '<tr>'
        for el in row:
            table += wrap_td(el)
        table += '</tr>'
    table += '</table>'
    return table

In [174]:

def limit_100_words(doc: str) -> str:
    words = doc.split(' ')
    if len(words) > 100:
        doc = ' '.join(words[:100]) + '…'
    return doc

def html_format_doc(doc: str) -> str:
    return doc.replace('<mask>', '<span style="color: lightgray; background-color: lightgray; font-weight: 100"> &lt;mask&gt;</span>')

def display_redacted_doc(idx):
    adv_doc = limit_100_words(df['perturbed_text'][idx])
    lex_doc = limit_100_words(dm.val_dataset[idx]['document_redact_lexical'])
    
    table = make_html_table(
        [
            ['Name', 'Lexically redacted', 'Adversarially redacted'],
            [dm.val_dataset[idx]['name'], html_format_doc(lex_doc), html_format_doc(adv_doc)]
        ]
    )
    
    display(HTML(table))

display_redacted_doc(81)

In [104]:
display_redacted_doc(801)

Name,Lexically redacted,Adversarially redacted
Truid Blaisse-terwindt,"<mask> <mask> ( 4 <mask> <mask> -- <mask> <mask> <mask> ) was a dutch female hockey - and tennis player who was active in the 1930s and 1940s . between 1935 and <mask> she participated in five wimbledon championships . her best result in the singles event was reaching the third round in <mask> , losing to dorothy round , and <mask> , losing to first seeded margaret du pont . in the doubles she reached the third round in <mask> and <mask> partnering compatriot madzy rollin couquerque . with ivo rinkel she reached the fourth round of the mixed doubles in <mask> . in…","<mask> <mask> ( <mask> <mask> <mask> -- <mask> <mask> <mask> ) was <mask> <mask> female <mask> - <mask> tennis player who <mask> <mask> in <mask> 1930s and 1940s . between 1935 and 1948 she participated in five wimbledon championships . her best result in <mask> singles event was reaching <mask> third round in 1937 , <mask> to dorothy round , and 1948 , losing to first <mask> margaret du <mask> . in the doubles she <mask> the third round in 1936 and <mask> <mask> compatriot madzy rollin couquerque . with ivo <mask> she <mask> the fourth round <mask> the mixed doubles in <mask> . in…"


In [145]:
display_redacted_doc(701)

Name,Lexically redacted,Adversarially redacted
Mikhail Bulgakov,"<mask> <mask> <mask> ( ; , ; -- <mask> <mask> , <mask> ) was a <mask> writer and <mask> active in the first half of the 20th century . he is best known for <mask> novel `` the master and margarita '' , which has been called one of the masterpieces of the 20th century .","<mask> <mask> <mask> ( ; , ; -- <mask> 10 , <mask> ) was a russian writer <mask> <mask> active in the first half of <mask> 20th century . he is best known for his <mask> `` the master and margarita '' , which has been called one of the masterpieces of the 20th century ."


In [168]:
import random
display_redacted_doc(random.choice(range(1000)))

Name,Lexically redacted,Adversarially redacted
Tony Do Pilar Patrao,"<mask> <mask> <mask> <mask> ( born <mask> , <mask> in <mask> , ) is a french-portuguese footballer . he currently plays in the cfa for <mask> <mask> . <mask> played at berre l'etang and <mask> . source : http://www.foot-national.com/4168-joueur-football- <mask>- <mask>- <mask>- <mask>.html","<mask> <mask> <mask> <mask> ( born 19 , <mask> <mask> <mask> , ) is a <mask> <mask> . he currently plays in the cfa <mask> <mask> <mask> . <mask> played <mask> berre l'etang and <mask> . source : http:// <mask>.foot-national.com/4168-joueur-football-do-pilar-patrao-tony.html"
